# Beta Data Analysis

We have been working to create a number of queries that describe the contents in the existing enriched-data collection. This notebook can be configured to point to any of the 3 environments (Live, Integration, DevQA).

In [1]:
# Related page
# http://confluence.epnet.com/display/ART/Spike+-+Date+field%28s%29+analyses+by+content+type+in+Raw-Meta

# Start of the script
import pandas as pd
import pymongo
from pymongo import MongoClient
import time
import datetime



env = input("Please Select an Environment D,I or L: ")
if env == "D":
    mng_svr = '10.25.216.142'
    mng_port = 27017
    mng_user = 'readOnlyUser'
    mng_pwd = 'uk53r8gb22zq76qb'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for DevQA")
elif env == "I":
    mng_svr = '10.25.152.243'
    mng_port = 27017
    mng_user = 'read'
    mng_pwd = 'QbSjdYLj6v'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for Integration")
elif env == "L":
    mng_svr = '10.26.152.183'
    mng_port = 27017
    mng_user = 'readOnlyUser'
    mng_pwd = 'uk53r8gb22zq76qb'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for Live")
else:
     print("No usable environment selected")
     exit()


#mng_conn_string = (mng_svr,mng_port,mng_user,mng_pwd)
print (mng_conn_string)


client = MongoClient (mng_svr,mng_port,username = mng_user, password = mng_pwd)
db = client.springer
collection = db['enriched-meta']



start = time.time()
print ("Connection complete")





#=====================================================
#Determine the content types in the collection

#Dist = collection.aggregate([{"$group" : {"_id" : {"type" : "content-type"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
#df = pd.DataFrame(list(Dist))



#Dist = collection.aggregate([{"$group" : {"_id" : {"vertical" : "$header.content-verticals"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
#df = pd.DataFrame(list(Dist))
#data = pd.DataFrame(list(collection.find().limit(1000)))

# End of the script

Please Select an Environment D,I or L: I
Configuring for Integration
('10.25.152.243', 27017, 'read', 'QbSjdYLj6v')
Connection complete


In [2]:
print("Initial analysis starting")
allRecords = collection.count_documents({})
print("Number of documents: ", allRecords)
#mfsRecords = collection.count_documents({"header.content-type":"sch"})
#print("Number of MFS documents: ", mfsRecords)
#imgRecords = collection.count_documents({"header.content-type":"imh"})
#print("Number of Image documents: ", imgRecords)
#ersRecords = collection.count_documents({"header.content-type":"ers"})
#print("Number of ERS documents: ", ersRecords)
print("Initial analysis complete")

Initial analysis starting
Number of documents:  5875205
Initial analysis complete


In [3]:
#Determine the content types in the collection
print ("Content Type Analysis starting")
Dist = collection.aggregate([{"$group" : {"_id" : {"type" : "$header.content-type"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
df = pd.DataFrame(list(Dist))
df["Perc"] = df['total'] / allRecords * 100

print ("Content Type Analysis complete")

Content Type Analysis starting
Content Type Analysis complete


In [4]:
df.round(2)

,_id,total,Perc
0,{'type': ['sch']},811047,70.08
1,{'type': ['imh']},277109,23.95
2,{'type': ['ers']},69115,5.97


Date Analysis; We will look at the prevalance of string dates in MFS fields.


In [5]:
print("Date analysis starting")
mfsRecords_w_date = collection.count_documents({
    '$and' : [{ "header.content-type" :  "sch"},
            {"date.string-date" :  {"$exists" : True}}]
})

Dist = collection.aggregate([{"$match" : { "header.content-type":"sch"}}, {"$group" : { "_id" : "$date.string-date", "total" : {"$sum" : 1} }}, {"$sort": {"total":-1}}])
df = pd.DataFrame(list(Dist))
print("MFSDocuments_W_Date: ", mfsRecords_w_date )
print("Percentage of MFS documents with a String Date: ", mfsRecords_w_date/mfsRecords * 100)
print("Date analysis complete")



Date analysis starting
MFSDocuments_W_Date:  811047
Percentage of MFS documents with a String Date:  100.0
Date analysis complete


In [6]:
df

,_id,total
0,"[Oct2017, 1506816000000000]",33490
1,"[2005, 1104537600000000]",4355
2,"[8/1/2017, 1501545600000000]",2294
3,"[2003, 1041379200000000]",1934
4,"[Oct2005, 1128124800000000]",1650
5,"[3/1/2006, 1141171200000000]",1620
6,"[Sep2005, 1125532800000000]",1590
7,"[Feb2005, 1107216000000000]",1583
8,"[Apr2005, 1112313600000000]",1577
9,"[Mar2005, 1109635200000000]",1554


In [7]:
#Structured Date field analysis look for Ranges in the Structured date fields (Year, Month, Day)
print("Structured Date analysis starting")
mfsRecords_w_year = collection.count_documents({
    '$and' : [{ "header.content-type" :  "sch"},
    {"date.year" :  {"$exists" : True}}]
})
print("Percentage of MFS documents with a Year: ", mfsRecords_w_year/mfsRecords * 100)

imgRecords_w_year = collection.count_documents({
    '$and' : [{ "header.content-type" :  "imh"},
            {"date.year" :  {"$exists" : True}}]
})
print("Percentage of Image documents with a Year: ", imgRecords_w_year/imgRecords * 100)

ersRecords_w_year = collection.count_documents({
    '$and' : [{ "header.content-type" :  "ers"},
            {"date.year" :  {"$exists" : True}}]
})
print("Percentage of MFS documents with a Year: ", ersRecords_w_year/ersRecords * 100)



Structured Date analysis starting
Percentage of MFS documents with a Year:  100.0
Percentage of Image documents with a Year:  100.0
Percentage of MFS documents with a Year:  100.0


In [8]:
print("Year Min / Max  analysis starting")
#year_max = collection.aggregate([
#    {"$match": {"date.date-type": "pub-date" }},
#    {"$group": {"_id:" : "year" , "ym" : {"$max" : "$date.year"}}}])

#year_max = collection.find({"date.year" : {"$exists" : True}}, sort=[("date.year", 1)])["date.year"]

year_max = collection.aggregate([
  { '$unwind': '$date' },
  { '$group': { '_id': 'null', 'max': { '$max': '$date.year' } } },
  { '$project': { 'max': 1, '_id':1 } }
]) 

print("Highest Year in the dataset: ",list(year_max)[0]['max'])

year_min = collection.aggregate([
  { '$unwind': '$date' },
  { '$group': { '_id': 'null', 'min': { '$min': '$date.year' } } },
  { '$project': { 'min': 1, '_id':1 } }
]) 

print("Lowest Year in the dataset: ",list(year_min)[0]['min'])

print("Year Min / Max  analysis complete")

Year Min / Max  analysis starting
Highest Year in the dataset:  2022
Lowest Year in the dataset:  1750
Year Min / Max  analysis complete


In [9]:
#Content verticals distribution
print ("Content Vertical Analysis Starting")
Dist = collection.aggregate([{"$group" : 
                              {"_id" : {"vertical" : "$header.content-verticals"},
                               "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
df = pd.DataFrame(list(Dist))
df["Perc"] = df['total'] / allRecords * 100
print ("Content Vertical Analysis complete")

Content Vertical Analysis Starting
Content Vertical Analysis complete


In [10]:
df.round(2)

,_id,total,Perc
0,{'vertical': [['Article']]},489089,42.26
1,{'vertical': [['Image']]},277109,23.95
2,"{'vertical': [['Article', 'Scholarly Article']]}",128081,11.07
3,{'vertical': [['Reference Entry']]},69117,5.97
4,"{'vertical': [['Reference Entry', 'Article']]}",35183,3.04
5,{'vertical': [['Book Review']]},22137,1.91
6,"{'vertical': [['Book Review', 'Review']]}",21656,1.87
7,"{'vertical': [['Letter', 'Article']]}",17716,1.53
8,"{'vertical': [['Product Review', 'Review']]}",14615,1.26
9,{'vertical': [['Letter']]},7063,0.61


In [11]:
print ("AN Analysis Starting")
Dist = collection.find({"header.content-type" : "sch"},{"article-id-group.CreativeWork-id" : 1})
df = pd.DataFrame(list(Dist))
print ("AN Analysis complete")

AN Analysis Starting
AN Analysis complete


In [12]:
df

,_id,article-id-group
0,5bb21e59588e1b919b1bb62e,"[{'CreativeWork-id': '44278735'}, {}]"
1,5bb21e59588e1b919b1bb62f,"[{'CreativeWork-id': '9009170201'}, {}]"
2,5bb21e59588e1b919b1bb630,"[{'CreativeWork-id': '8932109'}, {}]"
3,5bb21e59588e1b919b1bb631,"[{'CreativeWork-id': '125639415'}, {}]"
4,5bb21e59588e1b919b1bb632,"[{'CreativeWork-id': '13863166'}, {}]"
5,5bb21e59588e1b919b1bb633,"[{'CreativeWork-id': '8802311'}, {}]"
6,5bb21e59588e1b919b1bb634,"[{'CreativeWork-id': '23696503'}, {}]"
7,5bb21e59588e1b919b1bb635,"[{'CreativeWork-id': '9604081156'}, {}]"
8,5bb21e59588e1b919b1bb636,"[{'CreativeWork-id': '5961'}, {}]"
9,5bb21e59588e1b919b1bb638,"[{'CreativeWork-id': '124773053'}, {}]"


In [13]:
ANMongo = []
for rec in df['article-id-group']:
    if ('CreativeWork-id' in rec[0]):
        ANMongo.append(rec[0]['CreativeWork-id'])
    #print(rec[0]['CreativeWork-id'])


In [14]:
import cx_Oracle

conn = cx_Oracle.connect('aws_reader/mfaws2@10.68.72.101:1521/mfaws2.epnet.com')
print (conn.version)
print ("AN Analysis Starting")
cursor = conn.cursor()
result = cursor.execute('SELECT AN FROM ARTICLE_SCH')
ANOracle = []

for record in result.fetchall():
    ANOracle.append(str(record[0]))

cursor.close()
conn.close()
print ("AN Analysis complete")


10.2.0.3.0
AN Analysis Starting
AN Analysis complete


In [15]:
print("Count of Article number")
print("From Oracle " + str(len(ANOracle)))
print("From Mongo " + str(len(ANMongo)))


Count of Article number
From Oracle 814273
From Mongo 811047


In [16]:
differens=list(set(ANOracle) - set(ANMongo))
print("How many article numbers was matched: " + str(len(ANOracle) - len(differens)))
df = pd.DataFrame(differens)

How many article numbers was matched: 811047


In [17]:
df

,0
0,19932415
1,132401561
2,132965214
3,19932846
4,19930956
5,132933696
6,111933819
7,132965198
8,132887058
9,3057702


In [18]:
print("How many article numbers wasn't compare from Oracle: ", len(differens))

How many article numbers wasn't compare from Oracle:  3226


In [19]:
differensDoesNotExist=list(set(ANMongo) - set(ANOracle))

In [20]:
print("How many article numbers wasn't compare from Oracle to Mongo: ", len(differensDoesNotExist))

How many article numbers wasn't compare from Oracle to Mongo:  0


In [21]:

end = time.time()
print("Elapsed time in seconds: " + "{:.{}f}".format( (end - start), 2 ))

Elapsed time in seconds: 168.42


In [22]:
print("Job Completed at: "+str(datetime.datetime.now()))

Job Completed at: 2018-11-20 16:11:07.536816
